In [ ]:
import pandas as pd
from difflib import get_close_matches

class SistemaAutorizacionReal:
    def __init__(self):
        """
        Configuración con datos de ejemplo (reemplazar con conexión real a SharePoint)
        """
        self.tablas_capitadas = {
            'Bogotá': {
                'Colsubsidio': pd.DataFrame({
                    'codigo': ['890282', '881201'],
                    'nombre': [
                        'CONSULTA DE PRIMERA VEZ POR ESPECIALISTA EN OTORRINOLARINGOLOGIA',
                        'CONSULTA PRIORITARIA'
                    ],
                    'tipo': ['consulta', 'consulta']
                }),
                'Subred Norte': pd.DataFrame({
                    'codigo': ['834010'],
                    'nombre': ['CIRUGÍA MENOR'],
                    'tipo': ['procedimiento']
                })
            },
            'Soacha': {
                'Cafam': pd.DataFrame({
                    'codigo': ['899001'],
                    'nombre': ['ACETAMINOFEN+CODEÍNA'],
                    'tipo': ['medicamento']
                })
            }
        }

    def buscar_en_tablas(self, ciudad, ips, servicio):
        """Busca el servicio en los DataFrames cargados"""
        ciudad = ciudad.capitalize()

        if ciudad not in self.tablas_capitadas:
            return {'encontrado': False}

        # Buscar IPS con coincidencia aproximada
        ips_coincidente = None
        for ips_tabla in self.tablas_capitadas[ciudad].keys():
            if ips.lower() in ips_tabla.lower():
                ips_coincidente = ips_tabla
                break

        if not ips_coincidente:
            return {'encontrado': False}

        df = self.tablas_capitadas[ciudad][ips_coincidente]

        # Buscar por código o nombre
        if str(servicio).isdigit():
            resultado = df[df['codigo'] == str(servicio)]
        else:
            resultado = df[df['nombre'].str.contains(str(servicio), case=False)]

        if not resultado.empty:
            return {
                'encontrado': True,
                'datos': resultado.iloc[0].to_dict()
            }
        return {'encontrado': False}

    def iniciar_proceso(self):
        """Flujo interactivo con validaciones reales"""
        print("\n=== SISTEMA DE VALIDACIÓN REAL ===")

        # Paso 1: Ciudad
        ciudad = input("\n1. Ciudad de residencia (Bogotá/Soacha): ").strip()

        # Paso 2: IPS
        ips = input("2. IPS primaria (Colsubsidio/Subred Norte/Cafam): ").strip()

        # Paso 3: Servicio
        servicio = input("3. Código CUPS o nombre (ej: 890282, OTORRINOLARINGOLOGIA): ").strip()

        # Validar
        resultado = self.buscar_en_tablas(ciudad, ips, servicio)

        if resultado['encontrado']:
            print("\n=== RESULTADO ===")
            print("Servicio prestado directamente")
            print(f"Detalles: {resultado['datos']}")
        else:
            # Paso 4 y 5 solo si no se encuentra
            ips_remite = input("\n4. IPS que remite la orden: ").strip()
            diagnostico = input("5. Diagnóstico de la orden médica: ").strip()

            print("\n=== RESULTADO ===")
            print("Servicio NO encontrado en tablas de prestación directa")
            print("Se requiere autorización manual con los datos ingresados")

# Ejecutar
if __name__ == "__main__":
    sistema = SistemaAutorizacionReal()
    sistema.iniciar_proceso()

In [ ]:
import pandas as pd
import requests
from io import BytesIO
from difflib import get_close_matches

class SistemaAutorizacionPublicSheets:
    def __init__(self):
        """
        Configuración de los links públicos de Google Sheets.
        Ahora cada URL puede contener múltiples hojas.
        """
        self.sheets_publicas = {
            'PGP_CAFAM_COLSUBSIDIO': 'https://docs.google.com/spreadsheets/d/17pEopDY47lPmTuELX5MNYI--PgrtPgIR/export?format=xlsx',
            'CAPITA_MEDICAMENTOS_COLSUBSIDIO': 'https://docs.google.com/spreadsheets/d/14oSxL89ltBz1EMmmYeKWyvXgUoPqTw7u/export?format=xlsx',
            'CAPITA_BOGOTA_NORTE': 'https://docs.google.com/spreadsheets/d/1IFjQQ4ZXTXUuidli3v7k8Noa_KOgJMJF/export?format=xlsx',
            'CAPITA_BOGOTA_CENTRO_ORIENTE': 'https://docs.google.com/spreadsheets/d/111KwBjrlBGrFuG8NZVnSPkhX2kq2cXwF/export?format=xlsx',
            'CAPITA_BOGOTA_SUROCCIDENTE': 'https://docs.google.com/spreadsheets/d/1TeDQN-4EcAIiPnU1SfnG4xnl7oKBlXMx/export?format=xlsx',
            'CAPITA_MEDICAMENTOS_CAFAM': 'https://docs.google.com/spreadsheets/d/111KwBjrlBGrFuG8NZVnSPkhX2kq2cXwF/export?format=xlsx',
            'CAPITA_INSUMOS_CAFAM': 'https://docs.google.com/spreadsheets/d/1GYjV1knDZgQeTK8yxjKdAi2jb3Tifmvm/export?format=xlsx',
            'IPS_NO_ADSCRITAS': 'https://docs.google.com/spreadsheets/d/1Pt_dlfyuuiJG8XaPIkqLpdlRUM0Wz2dG/export?format=xlsx',
            'OTRA_TABLA': 'https://docs.google.com/spreadsheets/d/11YFUzjJQ0QON5z1Xf1oeK5MoGtvkHdPE/export?format=xlsx'
        }

        self.tablas = {}  # Ahora almacena diccionarios {nombre_hoja: df}
        self.cargar_tablas_publicas()

    def cargar_tabla_publica(self, url):
        """Carga un Google Sheet y devuelve un diccionario con todas sus hojas."""
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            excel_file = BytesIO(response.content)

            # Leer todas las hojas
            xls = pd.ExcelFile(excel_file)
            return {sheet_name: pd.read_excel(excel_file, sheet_name=sheet_name) for sheet_name in xls.sheet_names}

        except requests.exceptions.RequestException as e:
            print(f"Error de conexión al cargar {url}: {str(e)}")
            return None
        except Exception as e:
            print(f"Error al procesar {url}: {str(e)}")
            return None

    def cargar_tablas_publicas(self):
        """Carga todas las tablas públicas (ahora con todas sus hojas)."""
        print("Cargando tablas públicas desde Google Sheets...")
        for nombre, url in self.sheets_publicas.items():
            hojas = self.cargar_tabla_publica(url)
            if hojas:
                self.tablas[nombre] = hojas
                total_registros = sum(len(df) for df in hojas.values())
                print(f" \u2713 {nombre} cargado ({len(hojas)} hojas, {total_registros} registros)")
            else:
                print(f"\u274C Falló al cargar {nombre}")

    def buscar_servicio(self, ciudad, ips, servicio):
        """
        Busca en TODAS las hojas de las tablas relevantes.
        - Si servicio es numérico: busca en columnas 'CODIGO' o similares.
        - Si es texto: busca en cualquier campo.
        """
        ciudad = ciudad.upper()
        ips = ips.upper()
        servicio = str(servicio).strip()

        # 1. Determinar tablas relevantes según ciudad e IPS
        tablas_relevantes = []
        if 'BOGOT' in ciudad:
            if 'COLSUBSIDIO' in ips:
                tablas_relevantes.extend(['PGP_CAFAM_COLSUBSIDIO', 'CAPITA_MEDICAMENTOS_COLSUBSIDIO'])
            if 'NORTE' in ips:
                tablas_relevantes.append('CAPITA_BOGOTA_NORTE')
            if 'CENTRO' in ips or 'ORIENTE' in ips:
                tablas_relevantes.append('CAPITA_BOGOTA_CENTRO_ORIENTE')
            if 'SUROCCIDENTE' in ips:
                tablas_relevantes.append('CAPITA_BOGOTA_SUROCCIDENTE')
        elif 'SOACHA' in ciudad and 'CAFAM' in ips:
            tablas_relevantes.extend(['CAPITA_MEDICAMENTOS_CAFAM', 'CAPITA_INSUMOS_CAFAM'])

        # 2. Buscar en todas las hojas de las tablas relevantes
        for tabla_nombre in tablas_relevantes:
            if tabla_nombre in self.tablas:
                for hoja, df in self.tablas[tabla_nombre].items():
                    # Buscar por código (si es numérico y existe columna 'CODIGO')
                    if servicio.isdigit() and any('CODIGO' in col.upper() for col in df.columns):
                        col_codigo = [col for col in df.columns if 'CODIGO' in col.upper()][0]
                        resultado = df[df[col_codigo].astype(str).str.strip() == servicio]
                    else:
                        # Búsqueda por texto en todas las columnas
                        resultado = df[df.apply(lambda row: servicio.upper() in ' '.join(str(v) for v in row).upper(), axis=1)]

                    if not resultado.empty:
                        return {
                            'encontrado': True,
                            'datos': resultado.iloc[0].to_dict(),
                            'tabla': tabla_nombre,
                            'hoja': hoja,
                            'tipo': 'CAPITA' if 'CAPITA' in tabla_nombre else 'PGP'
                        }

        return {'encontrado': False}

    def iniciar_proceso(self):
        """Flujo interactivo del sistema con resultados mejorados."""
        print("\n=== SISTEMA DE AUTORIZACIÓN - VALIDACIÓN EN 5 PASOS ===")

        # Paso 1-3: Ciudad, IPS, Servicio
        ciudad = input("\n1. Ciudad de residencia (Bogotá/Soacha): ").strip()
        ips = input("2. IPS primaria (ej: Colsubsidio, Subred Norte): ").strip()
        servicio = input("3. Código CUPS o nombre del servicio: ").strip()

        # Buscar en tablas
        resultado = self.buscar_servicio(ciudad, ips, servicio)

        if resultado['encontrado']:
            print("\n=== RESULTADO ===")
            print(f"\u2713 **Servicio encontrado en:** {resultado['tabla']} (Hoja: {resultado['hoja']})")
            print(f"\U0001F539 **Tipo:** {resultado['tipo']}")
            print("\n**Detalles del servicio:**")
            for k, v in resultado['datos'].items():
                print(f"- {k}: {v}")
        else:
            # Paso 4-5: Si no se encuentra, pedir más datos
            print("\n=== RESULTADO ===")
            print("\U0001F534 Servicio NO encontrado en tablas de prestación directa.") #linea de unicode de bolita roja
            ips_remite = input("\n4. IPS que remite la orden: ").strip()
            diagnostico = input("5. Diagnóstico de la orden médica: ").strip()

            print("\n**Acciones requeridas:**")
            print(f"- Validar documentación con IPS {ips_remite}")
            print(f"- Escalar al comité de autorizaciones (Diagnóstico: {diagnostico})")

if __name__ == "__main__":
    # Instalar dependencias si no están instaladas
    try:
        import pandas as pd
        import requests
    except ImportError:
        print("Instalando dependencias necesarias...")
        import subprocess
        subprocess.run(['pip', 'install', 'pandas', 'requests'], check=True)

    sistema = SistemaAutorizacionPublicSheets()
    sistema.iniciar_proceso()

In [ ]:
import pandas as pd
import requests
from io import BytesIO
from difflib import get_close_matches

class SistemaAutorizacionPublicSheets:
    def __init__(self):
        """
        Configuración de los links públicos de Google Sheets.
        Ahora cada URL puede contener múltiples hojas.
        """
        self.sheets_publicas = {
            'PGP_CAFAM_COLSUBSIDIO': 'https://docs.google.com/spreadsheets/d/1IFjQQ4ZXTXUuidli3v7k8Noa_KOgJMJF/edit?gid=1931546774#gid=1931546774',
            'CAPITA_MEDICAMENTOS_COLSUBSIDIO': 'https://docs.google.com/spreadsheets/d/14oSxL89ltBz1EMmmYeKWyvXgUoPqTw7u/export?format=xlsx',
            'CAPITA_BOGOTA_NORTE': 'https://docs.google.com/spreadsheets/d/1IFjQQ4ZXTXUuidli3v7k8Noa_KOgJMJF/export?format=xlsx',
            'CAPITA_BOGOTA_CENTRO_ORIENTE': 'https://docs.google.com/spreadsheets/d/111KwBjrlBGrFuG8NZVnSPkhX2kq2cXwF/export?format=xlsx',
            'CAPITA_BOGOTA_SUROCCIDENTE': 'https://docs.google.com/spreadsheets/d/1TeDQN-4EcAIiPnU1SfnG4xnl7oKBlXMx/export?format=xlsx',
            'CAPITA_MEDICAMENTOS_CAFAM': 'https://docs.google.com/spreadsheets/d/111KwBjrlBGrFuG8NZVnSPkhX2kq2cXwF/export?format=xlsx',
            'CAPITA_INSUMOS_CAFAM': 'https://docs.google.com/spreadsheets/d/1GYjV1knDZgQeTK8yxjKdAi2jb3Tifmvm/export?format=xlsx',
            'IPS_NO_ADSCRITAS': 'https://docs.google.com/spreadsheets/d/1Pt_dlfyuuiJG8XaPIkqLpdlRUM0Wz2dG/export?format=xlsx',
            'OTRA_TABLA': 'https://docs.google.com/spreadsheets/d/11YFUzjJQ0QON5z1Xf1oeK5MoGtvkHdPE/export?format=xlsx'
        }

        self.tablas = {}  # Ahora almacena diccionarios {nombre_hoja: df}
        self.cargar_tablas_publicas()

    def cargar_tabla_publica(self, url):
        """Carga un Google Sheet y devuelve un diccionario con todas sus hojas."""
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            excel_file = BytesIO(response.content)

            # Leer todas las hojas
            xls = pd.ExcelFile(excel_file)
            return {sheet_name: pd.read_excel(excel_file, sheet_name=sheet_name) for sheet_name in xls.sheet_names}

        except requests.exceptions.RequestException as e:
            print(f"Error de conexión al cargar {url}: {str(e)}")
            return None
        except Exception as e:
            print(f"Error al procesar {url}: {str(e)}")
            return None

    def cargar_tablas_publicas(self):
        """Carga todas las tablas públicas (ahora con todas sus hojas)."""
        print("Cargando tablas públicas desde Google Sheets...")
        for nombre, url in self.sheets_publicas.items():
            hojas = self.cargar_tabla_publica(url)
            if hojas:
                self.tablas[nombre] = hojas
                total_registros = sum(len(df) for df in hojas.values())
                print(f"\u2713 {nombre} cargado ({len(hojas)} hojas, {total_registros} registros)")
            else:
                print(f"\u274C Falló al cargar {nombre}")

    def buscar_servicio(self, ciudad, ips, servicio):
        """
        Busca en TODAS las hojas de las tablas relevantes.
        - Si servicio es numérico: busca en columnas 'CODIGO' o similares.
        - Si es texto: busca en cualquier campo.
        """
        ciudad = ciudad.upper()
        ips = ips.upper()
        servicio = str(servicio).strip() #####

        # 1. Determinar tablas relevantes según ciudad e IPS
        tablas_relevantes = []
        if 'BOGOTA' in ciudad:
            if 'COLSUBSIDIO' in ips:
                tablas_relevantes.extend(['PGP_CAFAM_COLSUBSIDIO', 'CAPITA_MEDICAMENTOS_COLSUBSIDIO'])
            if 'NORTE' in ips:
                tablas_relevantes.append('CAPITA_BOGOTA_NORTE')
            if 'CENTRO' in ips or 'ORIENTE' in ips:
                tablas_relevantes.append('CAPITA_BOGOTA_CENTRO_ORIENTE')
            if 'SUROCCIDENTE' in ips:
                tablas_relevantes.append('CAPITA_BOGOTA_SUROCCIDENTE')
        elif 'SOACHA' in ciudad and 'CAFAM' in ips:
            tablas_relevantes.extend(['CAPITA_MEDICAMENTOS_CAFAM', 'CAPITA_INSUMOS_CAFAM'])

        # 2. Buscar en todas las hojas de las tablas relevantes
        for tabla_nombre in tablas_relevantes:
            if tabla_nombre in self.tablas:
                for hoja, df in self.tablas[tabla_nombre].items():
                    # Buscar por código (si es numérico y existe columna 'CODIGO')
                    if servicio.isdigit() and any('CODIGO' in col.upper() for col in df.columns):
                        col_codigo = [col for col in df.columns if 'CODIGO' in col.upper()][0]
                        resultado = df[df[col_codigo].astype(str).str.strip() == servicio]
                    else:
                        # Búsqueda por texto en todas las columnas
                        resultado = df[df.apply(lambda row: servicio.upper() in ' '.join(str(v) for v in row).upper(), axis=1)]

                    if not resultado.empty:
                        return {
                            'encontrado': True,
                            'datos': resultado.iloc[0].to_dict(),
                            'tabla': tabla_nombre,
                            'hoja': hoja,
                            'tipo': 'CAPITA' if 'CAPITA' in tabla_nombre else 'PGP'
                        }

        return {'encontrado': False}

    def iniciar_proceso(self):
        """Flujo interactivo del sistema con resultados mejorados."""
        print("\n=== SISTEMA DE AUTORIZACIÓN - VALIDACIÓN EN 5 PASOS ===")

        # Paso 1-3: Ciudad, IPS, Servicio
        ciudad = input("\n1. Ciudad de residencia (Bogotá/Soacha): ").strip()
        ips = input("2. IPS primaria (ej: Colsubsidio, Subred Norte): ").strip()
        servicio = input("3. Código CUPS o nombre del servicio: ").strip()

        # Buscar en tablas
        resultado = self.buscar_servicio(ciudad, ips, servicio)

        if resultado['encontrado']:
            print("\n=== RESULTADO ===")
            print(f"\u2713 **Servicio encontrado en:** {resultado['tabla']} (Hoja: {resultado['hoja']})")
            print(f"\U0001F539 **Tipo:** {resultado['tipo']}")
            print("\n**Detalles del servicio:**")
            for k, v in resultado['datos'].items():
                print(f"- {k}: {v}")
        else:
            # Paso 4-5: Si no se encuentra, pedir más datos
            print("\n=== RESULTADO ===")
            print("\U0001F534 Servicio NO encontrado en tablas de prestación directa.")
            ips_remite = input("\n4. IPS que remite la orden: ").strip()
            diagnostico = input("5. Diagnóstico de la orden médica: ").strip()

            print("\n**Acciones requeridas:**")
            print(f"- Validar documentación con IPS {ips_remite}")
            print(f"- Escalar al comité de autorizaciones (Diagnóstico: {diagnostico})")

if __name__ == "__main__":
    # Instalar dependencias si no están instaladas
    try:
        import pandas as pd
        import requests
    except ImportError:
        print("Instalando dependencias necesarias...")
        import subprocess
        subprocess.run(['pip', 'install', 'pandas', 'requests'], check=True)

    sistema = SistemaAutorizacionPublicSheets()
    sistema.iniciar_proceso()

In [3]:
import pandas as pd
import requests
from io import BytesIO
from difflib import get_close_matches

class SistemaAutorizacionPublicSheets:
    def __init__(self):
        """
        Configuración de los links públicos de Google Sheets.
        Ahora cada URL puede contener múltiples hojas.
        """
        self.sheets_publicas = {
    # Capita Bogotá Subred CentroOriente
    'CAPITA_BOGOTA_CENTRO_ORIENTE': 'https://docs.google.com/spreadsheets/d/1GYjV1knDZgQeTK8yxjKdAi2jb3Tifmvm/export?format=xlsx&gid=249228206',

    # Capita Bogotá Subred Norte
    'CAPITA_BOGOTA_NORTE': 'https://docs.google.com/spreadsheets/d/111KwBjrlBGrFuG8NZVnSPkhX2kq2cXwF/export?format=xlsx&gid=1078321027',

    # Capita Bogotá Subred Sur
    'CAPITA_BOGOTA_SUR': 'https://docs.google.com/spreadsheets/d/1TeDQN-4EcAIiPnU1SfnG4xnl7oKBlXMx/export?format=xlsx&gid=563951527',

    # Capita Bogotá Subred Suroccidente
    'CAPITA_BOGOTA_SUROCCIDENTE': 'https://docs.google.com/spreadsheets/d/1Pt_dlfyuuiJG8XaPIkqLpdlRUM0Wz2dG/export?format=xlsx&gid=1008155628',

    # Capita Insumos Cafam Colsubsidio Bogotá
    'CAPITA_INSUMOS_CAFAM_COLSUBSIDIO': 'https://docs.google.com/spreadsheets/d/14oSxL89ltBz1EMmmYeKWyvXgUoPqTw7u/export?format=xlsx&gid=856532779',

    # Capita Medicamentos Droguerías Cafam
    'CAPITA_MEDICAMENTOS_DROGUERIAS_CAFAM': 'https://docs.google.com/spreadsheets/d/17pEopDY47lPmTuELX5MNYI--PgrtPgIR/export?format=xlsx&gid=1138690448',

    # Exclusiones Medicamentos Capita Cafam y Colsubsidio
    'EXCLUSIONES_MEDICAMENTOS_CAPITA': 'https://docs.google.com/spreadsheets/d/1AaAgFYrTwt2jh8oS_OQ4eaHxMvVg6mrr/export?format=xlsx&gid=596289938',

    # Exclusiones Medicamentos Colsubsidio
    'EXCLUSIONES_MEDICAMENTOS_COLSUBSIDIO': 'https://docs.google.com/spreadsheets/d/1oPRFNo42XN-YOtgypqe3HCbFrYWCiZWb/export?format=xlsx&gid=1959011796',

    # Medicamentos Capita Colsubsidio
    'MEDICAMENTOS_CAPITA_COLSUBSIDIO': 'https://docs.google.com/spreadsheets/d/11YFUzjJQ0QON5z1Xf1oeK5MoGtvkHdPE/export?format=xlsx&gid=276903866',

    # Capita Cafam y Colsubsidio Bogotá y Soacha
    'CAPITA_CAFAM_COLSUBSIDIO': 'https://docs.google.com/spreadsheets/d/1IFjQQ4ZXTXUuidli3v7k8Noa_KOgJMJF/export?format=xlsx&gid=1931546774'
}

In [ ]:
import pandas as pd
import requests
from io import BytesIO

class SistemaAutorizacionSimple:
    def __init__(self):
        # URLs de las hojas de cálculo (formato exportación)
        self.urls = {
            'CENTRO_ORIENTE': 'https://docs.google.com/spreadsheets/d/1GYjV1knDZgQeTK8yxjKdAi2jb3Tifmvm/export?format=xlsx',
            'NORTE': 'https://docs.google.com/spreadsheets/d/111KwBjrlBGrFuG8NZVnSPkhX2kq2cXwF/export?format=xlsx',
            'SUR': 'https://docs.google.com/spreadsheets/d/1TeDQN-4EcAIiPnU1SfnG4xnl7oKBlXMx/export?format=xlsx',
            'SUROCCIDENTE': 'https://docs.google.com/spreadsheets/d/1Pt_dlfyuuiJG8XaPIkqLpdlRUM0Wz2dG/export?format=xlsx',
            'INSUMOS': 'https://docs.google.com/spreadsheets/d/14oSxL89ltBz1EMmmYeKWyvXgUoPqTw7u/export?format=xlsx',
            'MEDICAMENTOS': 'https://docs.google.com/spreadsheets/d/17pEopDY47lPmTuELX5MNYI--PgrtPgIR/export?format=xlsx',
            'EXCLUSIONES': 'https://docs.google.com/spreadsheets/d/1AaAgFYrTwt2jh8oS_OQ4eaHxMvVg6mrr/export?format=xlsx',
            'COLSUBSIDIO': 'https://docs.google.com/spreadsheets/d/11YFUzjJQ0QON5z1Xf1oeK5MoGtvkHdPE/export?format=xlsx',
            'CAFAM': 'https://docs.google.com/spreadsheets/d/1IFjQQ4ZXTXUuidli3v7k8Noa_KOgJMJF/export?format=xlsx'
        }

        # Carga los datos ignorando ocultos
        self.datos = self._cargar_datos()

    def _es_oculto(self, nombre):
        """Determina si un elemento es oculto (hoja o columna)"""
        if not isinstance(nombre, str):
            return False
        nombre = nombre.lower()
        oculto = any(palabra in nombre for palabra in [
            'hide', 'hidden', 'oculto', 'tmp', 'backup', '_',
            'old', 'draft', 'calculos', 'config'
        ])
        return oculto

    def _cargar_datos(self):
        """Carga los datos filtrando todo lo oculto"""
        datos = {}

        for nombre, url in self.urls.items():
            try:
                # Descargar archivo
                response = requests.get(url, timeout=30)
                excel = pd.ExcelFile(BytesIO(response.content))

                hojas = {}
                for hoja in excel.sheet_names:
                    # Saltar hojas ocultas
                    if self._es_oculto(hoja):
                        continue

                    # Leer datos
                    df = pd.read_excel(excel, sheet_name=hoja)

                    # Filtrar columnas ocultas
                    columnas_visibles = [col for col in df.columns if not self._es_oculto(col)]
                    df = df[columnas_visibles]

                    # Guardar solo si tiene datos
                    if not df.empty:
                        hojas[hoja] = df

                if hojas:
                    datos[nombre] = hojas

            except Exception as e:
                print(f"Error al cargar {nombre}: {str(e)}")

        return datos

    def mostrar_datos(self):
        """Muestra qué datos se cargaron"""
        for tabla, hojas in self.datos.items():
            print(f"\nTabla: {tabla}")
            for hoja, df in hojas.items():
                print(f"  Hoja: {hoja} ({len(df)} registros)")
                print(f"  Columnas: {list(df.columns)}")

# Uso del sistema
if __name__ == "__main__":
    print("Cargando datos...")
    sistema = SistemaAutorizacionSimple()
    sistema.mostrar_datos()

In [ ]:
import pandas as pd
import requests
from io import BytesIO
from difflib import get_close_matches

class SistemaAutorizacionMejorado:
    def __init__(self):
        self.sheets_publicas = {
            'CAPITA_BOGOTA_CENTRO_ORIENTE': 'https://docs.google.com/spreadsheets/d/1GYjV1knDZgQeTK8yxjKdAi2jb3Tifmvm/export?format=xlsx',
            'CAPITA_BOGOTA_NORTE': 'https://docs.google.com/spreadsheets/d/111KwBjrlBGrFuG8NZVnSPkhX2kq2cXwF/export?format=xlsx',
            'CAPITA_BOGOTA_SUR': 'https://docs.google.com/spreadsheets/d/1TeDQN-4EcAIiPnU1SfnG4xnl7oKBlXMx/export?format=xlsx',
            'CAPITA_BOGOTA_SUROCCIDENTE': 'https://docs.google.com/spreadsheets/d/1Pt_dlfyuuiJG8XaPIkqLpdlRUM0Wz2dG/export?format=xlsx',
            'CAPITA_INSUMOS_CAFAM_COLSUBSIDIO': 'https://docs.google.com/spreadsheets/d/14oSxL89ltBz1EMmmYeKWyvXgUoPqTw7u/export?format=xlsx',
            'CAPITA_MEDICAMENTOS_DROGUERIAS_CAFAM': 'https://docs.google.com/spreadsheets/d/17pEopDY47lPmTuELX5MNYI--PgrtPgIR/export?format=xlsx',
            'EXCLUSIONES_MEDICAMENTOS_CAPITA': 'https://docs.google.com/spreadsheets/d/1AaAgFYrTwt2jh8oS_OQ4eaHxMvVg6mrr/export?format=xlsx',
            'MEDICAMENTOS_CAPITA_COLSUBSIDIO': 'https://docs.google.com/spreadsheets/d/11YFUzjJQ0QON5z1Xf1oeK5MoGtvkHdPE/export?format=xlsx',
            'CAPITA_CAFAM_COLSUBSIDIO': 'https://docs.google.com/spreadsheets/d/1IFjQQ4ZXTXUuidli3v7k8Noa_KOgJMJF/export?format=xlsx'
        }

        # Columnas objetivo con variaciones posibles
        self.columnas_objetivo = {
            'MEDICAMENTOS': ['medicamento', 'medicamentos', 'nombre medicamento'],
            'DESCRIPCION_INSUMOS': ['descripción insumos', 'descripcion insumos', 'detalle insumos'],
            'CODIGO_CUPS': ['código cups', 'codigo cups', 'cups', 'cod cups'],
            'DESCRIPCION_SERVICIO': ['descripción servicio', 'descripcion servicio', 'detalle servicio'],
            'DESCRIPCION_MEDICAMENTOS': ['descripción medicamento', 'descripcion medicamento', 'detalle medicamento']
        }

        self.tablas = {}
        self.cargar_tablas()

    def encontrar_columna(self, df):
        """Encuentra las columnas objetivo con coincidencia flexible"""
        columnas_encontradas = {}

        for col in df.columns:
            col_lower = str(col).lower().strip()

            # Buscar coincidencias para cada tipo de columna
            for tipo, variantes in self.columnas_objetivo.items():
                for variante in variantes:
                    if variante in col_lower:
                        columnas_encontradas[tipo] = col
                        break

        return columnas_encontradas

    def cargar_tablas(self):
        """Carga las tablas con detección mejorada de columnas"""
        print("Cargando tablas con búsqueda flexible de columnas...")

        for nombre, url in self.sheets_publicas.items():
            try:
                response = requests.get(url, timeout=20)
                excel_file = BytesIO(response.content)
                xls = pd.ExcelFile(excel_file)

                hojas_validas = {}
                for hoja in xls.sheet_names:
                    df = pd.read_excel(excel_file, sheet_name=hoja)

                    # Encontrar columnas objetivo
                    columnas = self.encontrar_columna(df)

                    if columnas:
                        # Crear nuevo DataFrame solo con las columnas encontradas
                        df_filtrado = df[list(columnas.values())].copy()
                        df_filtrado.columns = columnas.keys()  # Estandarizar nombres
                        df_filtrado = df_filtrado.dropna(how='all')

                        if not df_filtrado.empty:
                            hojas_validas[hoja] = df_filtrado

                if hojas_validas:
                    self.tablas[nombre] = hojas_validas
                    print(f"\u2713 {nombre}: Columnas encontradas - {list(next(iter(hojas_validas.values())).columns)}")
                else:
                    print(f"\u26A0 {nombre}: No se detectaron las columnas objetivo. Revisar nombres.")
                    # Ensure the print statement is correctly indented within the else block
                    print(f"   Columnas disponibles en la primera hoja: {list(pd.read_excel(excel_file, sheet_name=xls.sheet_names[0]).columns)}")

            # Ensure the except block is correctly indented relative to the try block
            except Exception as e:
                print(f"\u274C Error cargando {nombre}: {str(e)}")


    def buscar_servicio(self, texto):
        """Busca en todas las tablas cargadas"""
        texto = str(texto).strip().lower()
        resultados = []

        for tabla, hojas in self.tablas.items():
            for hoja, df in hojas.items():
                for col in df.columns:
                    try:
                        mask = df[col].astype(str).str.lower().str.contains(texto)
                        if mask.any():
                            resultados.append({
                                'tabla': tabla,
                                'hoja': hoja,
                                'columna': col,
                                'registros': df[mask].to_dict('records')
                            })
                    except:
                        continue

        return resultados

    def iniciar_busqueda(self):
        """Interfaz de búsqueda mejorada"""
        print("\n=== SISTEMA DE AUTORIZACIONES ===")
        print("Las columnas objetivo son:")
        for col in self.columnas_objetivo.keys():
            print(f"- {col} (variantes: {', '.join(self.columnas_objetivo[col])})")

        while True:
            busqueda = input("\nIngrese código/nombre o 'salir': ").strip()
            if busqueda.lower() == 'salir':
                break

            resultados = self.buscar_servicio(busqueda)

            if resultados:
                print(f"\U0001F50D {len(resultados)} coincidencias:") #Si te genera error agrega /n antes del codigo unicode de lupa
                for i, res in enumerate(resultados, 1):
                    print(f"\n{i}. Tabla: {res['tabla']} | Hoja: {res['hoja']}")
                    print(f"   Columna: {res['columna']}")
                    print("   Primer resultado:", res['registros'][0])
            else:
                print("\nNo se encontraron coincidencias")

if __name__ == "__main__":
    sistema = SistemaAutorizacionMejorado()
    sistema.iniciar_busqueda()

In [ ]:
import pandas as pd
import requests
from io import BytesIO
from difflib import get_close_matches

class SistemaAutorizacionMejorado:
    def __init__(self):
        self.sheets_publicas = {
            'CAPITA_BOGOTA_CENTRO_ORIENTE': 'https://docs.google.com/spreadsheets/d/1GYjV1knDZgQeTK8yxjKdAi2jb3Tifmvm/export?format=xlsx',
            'CAPITA_BOGOTA_NORTE': 'https://docs.google.com/spreadsheets/d/111KwBjrlBGrFuG8NZVnSPkhX2kq2cXwF/export?format=xlsx',
            'CAPITA_BOGOTA_SUR': 'https://docs.google.com/spreadsheets/d/1TeDQN-4EcAIiPnU1SfnG4xnl7oKBlXMx/export?format=xlsx',
            'CAPITA_BOGOTA_SUROCCIDENTE': 'https://docs.google.com/spreadsheets/d/1Pt_dlfyuuiJG8XaPIkqLpdlRUM0Wz2dG/export?format=xlsx',
            'CAPITA_INSUMOS_CAFAM_COLSUBSIDIO': 'https://docs.google.com/spreadsheets/d/14oSxL89ltBz1EMmmYeKWyvXgUoPqTw7u/export?format=xlsx',
            'CAPITA_MEDICAMENTOS_DROGUERIAS_CAFAM': 'https://docs.google.com/spreadsheets/d/17pEopDY47lPmTuELX5MNYI--PgrtPgIR/export?format=xlsx',
            'EXCLUSIONES_MEDICAMENTOS_CAPITA': 'https://docs.google.com/spreadsheets/d/1AaAgFYrTwt2jh8oS_OQ4eaHxMvVg6mrr/export?format=xlsx',
            'MEDICAMENTOS_CAPITA_COLSUBSIDIO': 'https://docs.google.com/spreadsheets/d/11YFUzjJQ0QON5z1Xf1oeK5MoGtvkHdPE/export?format=xlsx',
            'CAPITA_CAFAM_COLSUBSIDIO': 'https://docs.google.com/spreadsheets/d/1IFjQQ4ZXTXUuidli3v7k8Noa_KOgJMJF/export?format=xlsx'
        }

        # Columnas objetivo con variaciones posibles
        self.columnas_objetivo = {
            'MEDICAMENTOS': ['medicamento', 'medicamentos', 'nombre medicamento'],
            'DESCRIPCION_INSUMOS': ['descripción insumos', 'descripcion insumos', 'detalle insumos'],
            'CODIGO_CUPS': ['código cups', 'codigo cups', 'cups', 'cod cups'],
            'DESCRIPCION_SERVICIO': ['descripción servicio', 'descripcion servicio', 'detalle servicio'],
            'DESCRIPCION_MEDICAMENTOS': ['descripción medicamento', 'descripcion medicamento', 'detalle medicamento']
        }

        self.tablas = {}
        self.cargar_tablas()

    def encontrar_columna(self, df):
        """Encuentra las columnas objetivo con coincidencia flexible"""
        columnas_encontradas = {}

        for col in df.columns:
            col_lower = str(col).lower().strip()

            # Buscar coincidencias para cada tipo de columna
            for tipo, variantes in self.columnas_objetivo.items():
                for variante in variantes:
                    if variante in col_lower:
                        columnas_encontradas[tipo] = col
                        break

        return columnas_encontradas

    def cargar_tablas(self):
        """Carga las tablas con detección mejorada de columnas"""
        print("Cargando tablas con búsqueda flexible de columnas...")

        for nombre, url in self.sheets_publicas.items():
            try:
                response = requests.get(url, timeout=20)
                excel_file = BytesIO(response.content)
                xls = pd.ExcelFile(excel_file)

                hojas_validas = {}
                for hoja in xls.sheet_names:
                    df = pd.read_excel(excel_file, sheet_name=hoja)

                    # Encontrar columnas objetivo
                    columnas = self.encontrar_columna(df)

                    if columnas:
                        # Crear nuevo DataFrame solo con las columnas encontradas
                        df_filtrado = df[list(columnas.values())].copy()
                        df_filtrado.columns = columnas.keys()  # Estandarizar nombres
                        df_filtrado = df_filtrado.dropna(how='all')

                        if not df_filtrado.empty:
                            hojas_validas[hoja] = df_filtrado

                if hojas_validas:
                    self.tablas[nombre] = hojas_validas
                    print(f"\u2713 {nombre}: Columnas encontradas - {list(next(iter(hojas_validas.values())).columns)}")
                else:
                    print(f"\u26A0 {nombre}: No se detectaron las columnas objetivo. Revisar nombres.")
                    print(f"   Columnas disponibles en la primera hoja: {list(pd.read_excel(excel_file, sheet_name=xls.sheet_names[0]).columns)}")

            except Exception as e:
                print(f"\u274C Error cargando {nombre}: {str(e)}")

    def buscar_servicio(self, ciudad, ips, servicio):
        """Busca en todas las tablas cargadas según ciudad e IPS"""
        ciudad = ciudad.upper()
        ips = ips.upper()
        servicio = str(servicio).strip().lower()
        resultados = []

        # Determinar tablas relevantes según ciudad e IPS
        tablas_relevantes = []
        if 'BOGOT' in ciudad:
            if 'COLSUBSIDIO' in ips:
                tablas_relevantes.extend(['CAPITA_CAFAM_COLSUBSIDIO', 'MEDICAMENTOS_CAPITA_COLSUBSIDIO'])
            if 'NORTE' in ips:
                tablas_relevantes.append('CAPITA_BOGOTA_NORTE')
            if 'CENTRO' in ips or 'ORIENTE' in ips:
                tablas_relevantes.append('CAPITA_BOGOTA_CENTRO_ORIENTE')
            if 'SUROCCIDENTE' in ips:
                tablas_relevantes.append('CAPITA_BOGOTA_SUROCCIDENTE')
        elif 'SOACHA' in ciudad and 'CAFAM' in ips:
            tablas_relevantes.extend(['CAPITA_MEDICAMENTOS_DROGUERIAS_CAFAM', 'CAPITA_INSUMOS_CAFAM_COLSUBSIDIO'])

        # Buscar en las tablas relevantes
        for tabla_nombre in tablas_relevantes:
            if tabla_nombre in self.tablas:
                for hoja, df in self.tablas[tabla_nombre].items():
                    for col in df.columns:
                        try:
                            mask = df[col].astype(str).str.lower().str.contains(servicio)
                            if mask.any():
                                resultados.append({
                                    'encontrado': True,
                                    'datos': df[mask].iloc[0].to_dict(),
                                    'tabla': tabla_nombre,
                                    'hoja': hoja,
                                    'tipo': 'CAPITA' if 'CAPITA' in tabla_nombre else 'PGP'
                                })
                                return resultados[0]  # Devolver el primer resultado encontrado
                        except:
                            continue

        return {'encontrado': False}

    def iniciar_proceso(self):
        """Flujo interactivo del sistema con los 5 pasos"""
        print("\n=== SISTEMA DE AUTORIZACIÓN - VALIDACIÓN EN 5 PASOS ===")

        # Paso 1-3: Ciudad, IPS, Servicio
        ciudad = input("\n1. Ciudad de residencia (Bogotá/Soacha): ").strip()
        ips = input("2. IPS primaria (ej: Colsubsidio, Subred Norte): ").strip()
        servicio = input("3. Código CUPS o nombre del servicio: ").strip()

        # Buscar en tablas
        resultado = self.buscar_servicio(ciudad, ips, servicio)

        if resultado.get('encontrado', False):
            print("\n=== RESULTADO ===")
            print(f"\u2713 **Servicio encontrado en:** {resultado['tabla']} (Hoja: {resultado['hoja']})")
            print(f"\U0001F539 **Tipo:** {resultado['tipo']}")
            print("\n**Detalles del servicio:**")
            for k, v in resultado['datos'].items():
                print(f"- {k}: {v}")
        else:
            # Paso 4-5: Si no se encuentra, pedir más datos
            print("\n=== RESULTADO ===")
            print("\U0001F534 Servicio NO encontrado en tablas de prestación directa.")
            ips_remite = input("\n4. IPS que remite la orden: ").strip()
            diagnostico = input("5. Diagnóstico de la orden médica: ").strip()

            print("\n**Acciones requeridas:**")
            print(f"- Validar documentación con IPS {ips_remite}")
            print(f"- Escalar al comité de autorizaciones (Diagnóstico: {diagnostico})")

if __name__ == "__main__":
    sistema = SistemaAutorizacionMejorado()
    sistema.iniciar_proceso()